In [1]:
(do 
    (require `serpent.core)
    (require `serpent.pprint)
    (require `serpent.jupyter)
    (serpent.core/configure)
    (add-dependencies [org.mitre.tdp/ttfs-workflow "3.1.25" :exclusions [[net.sf.jsi/jsi]]])
    (set-cda-config :root "tdp:3.1"))

Retrieving org/mitre/caasd/caasd-project/0.1.3-SNAPSHOT/maven-metadata.xml (1k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-snapshots/
Retrieving org/mitre/caasd/jhlabs/proj4-library/1.0.6.1-SNAPSHOT/maven-metadata.xml (1k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-snapshots/


{:configuration
 #object[org.apache.hadoop.conf.Configuration 0x65f65cdd "\"Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml\""],
 :meta-repository
 #object[org.mitre.cda.hbase.HBaseMetaRepository 0x1862feab "\"HBaseMetaRepository[tdp:3.1-meta]\""],
 :hbase-repository
 #object[org.mitre.cda.core.CachingKeyDereferencer 0x27979b92 "\"org.mitre.cda.core.CachingKeyDereferencer@27979b92\""]}


In [2]:
(add-dependencies [org.mitre.tdp/ttfs-procedure "3.1.25" :exclusions [[net.sf.jsi/jsi]]])
(add-dependencies [org.mitre.tdp/tdp-infrastructure "3.1.25" :exclusions [[net.sf.jsi/jsi]]])

("com.google.guava:guava:23.0"
 "net.java.dev.jets3t:jets3t:0.9.4"
 "org.json:json:20140107"
 "org.apache.commons:commons-math3:3.4.1"
 "org.codehaus.mojo:animal-sniffer-annotations:1.14"
 "com.google.errorprone:error_prone_annotations:2.0.18"
 "org.codehaus.jackson:jackson-xc:1.9.13"
 "org.apache.commons:commons-math:2.2"
 "org.scala-lang:scala-reflect:2.11.8"
 "org.apache.httpcomponents:httpcore:4.4.1"
 "com.google.j2objc:j2objc-annotations:1.1"
 "commons-cli:commons-cli:1.2"
 "com.google.code.findbugs:jsr305:1.3.9"
 "org.objenesis:objenesis:2.1"
 "joda-time:joda-time:2.9.4"
 "com.jamesmurty.utils:java-xmlbuilder:1.1"
 "net.iharder:base64:2.3.8"
 "commons-logging:commons-logging:1.2"
 "commons-codec:commons-codec:1.9"
 "org.scala-lang:scala-library:2.11.8"
 ...)


In [3]:
(import org.mitre.caasd.ttfs.ArincTransition
        org.mitre.caasd.ttfs.ArincFix
        org.mitre.caasd.ttfs.ArincAirway
        org.mitre.caasd.ttfs.ArincAirport)

org.mitre.caasd.ttfs.ArincAirport


In [4]:
;; data is small we can just materialize an entire cycle
(def cyc 20210101)

(def transitions (load-data ArincTransition :dates cyc :only "CIFP"))
(def fixes (load-data ArincFix :dates cyc :only "CIFP"))
(def airways (load-data ArincAirway :dates cyc :only "CIFP"))
(def airports (load-data ArincAirport :dates cyc :only "CIFP"))

(str "Transitions: " (count transitions) " Fixes: " (count fixes) " Airways: " (count airways) " Airports: " (count airports))

"Transitions: 61338 Fixes: 67911 Airways: 1550 Airports: 13765"


### Helper Functions

In [5]:
;; Grab all the transitions
(defn null-or-equals [e v]
    (or (= nil v) (= e nil) (= e v)))

(defn proc-filter [pname & kwargs]
    (let [{:keys [airport source] :or {airport nil source nil} :as opts} kwargs]
        (fn [p] (and 
                    (null-or-equals airport ((comp :name :airport) p)) 
                    (null-or-equals pname (:procedure_name p))
                    (null-or-equals source (:navigation_source p))))))

(defn fix-filter [col]
    (fn [f] 
        (some #(= % ((comp :name :identification) f)) col)))

(defn airway-filter [awname & kwargs]
    (let [{:keys [entry exit area-code source]
           :or {entry nil
                exit nil
                area-code nil
                source nil}
           :as opts}
          kwargs]
        (fn [a] (and
                    (null-or-equals awname ((comp :name :identification) a))
                    (some #(null-or-equals entry ((comp :name :identification :path_terminator) %)) (:points a))
                    (some #(null-or-equals exit ((comp :name :identification :path_terminator) %)) (:points a))
                    (null-or-equals area-code ((comp :customer_area_code :arinc_record_info) a))
                    (null-or-equals source ((comp :source :arinc_record_info) a))))))

(defn peek? [ele f] (f ele) ele)

(defn cda-key->string [k] (.toString (.getPrimaryKey k)))

#'user/cda-key->string


### Airway Mocking

In [6]:
(def airwayy (->> airways (filter #(= "Q6" (:identifier %)))))

#'user/airwayy


In [7]:
(defn fixName [leg] (str ((comp :name :identification :path_terminator) leg)))
(defn fixLatLon [leg] (let [latitude ((comp :latitude :path_terminator) leg)
                            longitude ((comp :longitude :path_terminator) leg)]
                          (str " , " latitude " , " longitude)))
(defn toMockitoAirwayBody [airway]
    (let [body (reduce str (->> (:legs airway)
     (map #(str "Leg " (fixName %) "= TF( \"" (fixName %) "\"" (fixLatLon %) "); \n"))))]
    (str body " - primary_key " (cda-key->string (:primary_key airway)) "\n")))

#'user/toMockitoAirwayBody


In [8]:
(->> airwayy (map toMockitoAirwayBody))

("Leg TKA= TF( "TKA" , 62.29837777777777 , -150.10524999999998); 
Leg JOKAP= TF( "JOKAP" , 63.91291111111111 , -150.97479166666668); 
Leg KUTDE= TF( "KUTDE" , 66.32227499999999 , -152.48358055555553); 
Leg LACIL= TF( "LACIL" , 69.504975 , -155.00948055555557); 
Leg BRW= TF( "BRW" , 71.27342777777778 , -156.78811666666667); 
 - primary_key 4-2YeCFS_9AAAJZ
")


### Transition Mocking

#### Procedures By Airport

In [9]:
(def airport-transitions (->> transitions (filter #(= "KMCO" ((comp :name :airport) %)))))
(frequencies (map :procedure_name airport-transitions))

{"I36R" 3,
 "R35R" 4,
 "I35R" 3,
 "R36R" 4,
 "CWRLD4" 4,
 "COSTR3" 6,
 "L36R" 3,
 "LEESE2" 5,
 "R17L" 3,
 "L17R" 3,
 "JAG6" 1,
 "GOOFY7" 7,
 "I18R" 3,
 "BAIRN4" 4,
 "L35R" 3,
 "I35L" 3,
 "I17L" 3,
 "R36L" 4,
 "R18R" 2,
 "PIGLT6" 3,
 "BITHO7" 1,
 "R17R" 3,
 "L17L" 3,
 "BUGGZ4" 3,
 "R35L" 4,
 "L18R" 3,
 "MINEE5" 5,
 "L35L" 3,
 "I17R" 3,
 "R18L" 3}


In [42]:
(def proc-trans (filter (proc-filter "I17R" :airport "KMCO" :source "CIFP") transitions))
(str "total transitions: " (count proc-trans))

"total transitions: 3"


In [43]:
(clojure.string/join "," (map (comp str get-key :primary_key) proc-trans))

"4mIH87B6x_AAC-A,4mIH87B6x_AAC-B,4mIH87B6x_AAC-O"


In [44]:
(map :primary_key (filter #(= "KMCO" ((comp :name :identification) %)) airports))

(Key<4pvX1Gw_G4AACR->)


In [45]:
(first (load-data org.mitre.caasd.ttfs.NfdcAirport :dates cyc :columns [:primary_key [:icao_identifier = "KMCO"]]))

Record
{:primary_key Key<4Fk9t23D4DAAARe>,
 :icao_identifier "KMCO"}


#### Mocking That Business

In [46]:
(defn toMockitoTransitionBody [transition]
    (reduce str (->> (:legs transition)
     (map #(str "Leg " (fixName %) "=" (:leg_type %) " ( \"" (fixName %) "\"" (fixLatLon %) "); \n")))))

(defn toMockitoTransitionMethod [transition] 
    (let [tname (:transition_name transition)
          pname (:procedure_name transition)
          aname ((comp :name :airport) transition)]
        (str "private static Transition " (:transition_name transition) "(){\n"
         (toMockitoTransitionBody transition)
         "return transition(\"" tname "\",\"" pname "\",\"" aname "\",,,);\n}")))

(defn mockitizeMeCaptain [transitions]
    (reduce str (->> transitions (map #(str (toMockitoTransitionMethod %) "\n")))))

#'user/mockitizeMeCaptain


In [47]:
(mockitizeMeCaptain proc-trans)

"private static Transition (){
Leg SACRO=IF ( "SACRO" , 28.74290277777778 , -81.29910555555556); 
Leg TACOT=CF ( "TACOT" , 28.690266666666666 , -81.29855277777777); 
Leg DALTY=CF ( "DALTY" , 28.637633333333333 , -81.298); 
Leg ELLAN=CF ( "ELLAN" , 28.58499722222222 , -81.29744722222222); 
Leg GLOSI=CF ( "GLOSI" , 28.55859722222222 , -81.29716944444445); 
Leg MINCO=CF ( "MINCO" , 28.511105555555556 , -81.29667222222223); 
Leg RW17R=CF ( "RW17R" , 28.43561111111111 , -81.29588055555556); 
return transition("","I17R","KMCO",,,);
}
private static Transition (){
Leg NOGGI=CF ( "NOGGI" , 28.163497222222222 , -81.29303888888889); 
Leg NOGGI=HM ( "NOGGI" , 28.163497222222222 , -81.29303888888889); 
return transition("","I17R","KMCO",,,);
}
private static Transition RATOY(){
Leg RATOY=IF ( "RATOY" , 28.82301111111111 , -81.29995); 
Leg SACRO=CF ( "SACRO" , 28.74290277777778 , -81.29910555555556); 
return transition("RATOY","I17R","KMCO",,,);
}
"
